In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import re
from importlib import reload
import warnings
from pprint import pprint
import string
import pyLDAvis.gensim as gensimvis
import pickle 
import pyLDAvis
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
import clean_tweets_dataframe as cld

In [ ]:
reload(cld)

In [ ]:
# from clean_tweets_dataframe import CleanTweets

In [ ]:
df_tweet=pd.read_csv("processed_tweet_data.csv")

In [ ]:
cleaner=cld.CleanTweets(df_tweet)

**Using cleaner module from clean_tweets_dataframe to clean the imported dataset**

In [ ]:
df_tweet=cleaner.drop_unwanted_column(df_tweet)
df_tweet=cleaner.drop_duplicate(df_tweet)
df_tweet=cleaner.convert_to_datetime(df_tweet)
df_tweet=cleaner.convert_to_numbers(df_tweet)
df_tweet=cleaner.remove_other_languages_tweets(df_tweet)
df_tweet=cleaner.treat_special_characters(df_tweet)

In [ ]:
print(df_tweet.shape)
df_tweet

In [10]:
df_tweet['lang'].value_counts()

en      619
fr      183
kiny     52
Name: lang, dtype: int64

###  Export french tweets, make translation and insert back new translated tweets

In [11]:
df_trans=df_tweet.query("lang=='fr'| lang =='kiny'")[['original_text','cleaned_text','lang']]
df_trans.to_csv('df_to_translate.csv')

In [12]:
# translation completed

In [13]:
dict_to_translate=df_tweet.query("lang=='fr'| lang =='kiny'")['cleaned_text'].to_dict()

dict_translated=pd.read_csv('df_translated.csv',index_col=0,usecols=[0,4]).to_dict()['translation']

In [14]:
for idx in dict_to_translate.keys():
    df_tweet.loc[idx, 'cleaned_text'] = dict_translated[idx]

In [15]:
from textblob import TextBlob

In [16]:
for idx in dict_translated.keys():
    blob = TextBlob(dict_translated[idx])
    pol=blob.sentiment.polarity
    df_tweet.loc[idx, 'sentiment'] = 'Positive' if pol>0 else ('Negative' if pol < 0 else 'Neutral')

#### Save new dataframe for dashboard creation

In [17]:
df_tweet.to_csv('sql_dashboard/processed_tweet_data.csv')

In [32]:
d_mostflwd=df_tweet[['original_author']]

In [33]:
d_mostflwd['hh']=d_mostflwd['original_author'].apply(lambda x : '['+x+']'+'(https://twitter.com/'+str(x)+')')

In [34]:
d_mostflwd

,original_author,hh
8,Nduwamungu5,[Nduwamungu5](https://twitter.com/Nduwamungu5)
9,EUwatchers,[EUwatchers](https://twitter.com/EUwatchers)
17,123_INFO_DE,[123_INFO_DE](https://twitter.com/123_INFO_DE)
18,AriaSoondingie,[AriaSoondingie](https://twitter.com/AriaSoond...
19,arson_cole,[arson_cole](https://twitter.com/arson_cole)
...,...,...
10344,ngoyikabongo1,[ngoyikabongo1](https://twitter.com/ngoyikabon...
10371,egonwillighagen,[egonwillighagen](https://twitter.com/egonwill...
10384,iamdedit,[iamdedit](https://twitter.com/iamdedit)
10394,steverukundo,[steverukundo](https://twitter.com/steverukundo)


### EDA of Tweets

In [ ]:
# Check the percentage of missing values in every column
ax = df_tweet.isna().sum().sort_values().plot(kind = 'barh', figsize = (7, 9))
plt.title('Percentage of Missing Values Per Column in Tweets data', fontdict={'size':15})

for p in ax.patches:
    percentage ='{:,.3f}%'.format((p.get_width()/df_tweet.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

In [ ]:
df_tweet['place'].value_counts(sort=True, ascending=False)[:10].plot(kind='barh',
                                                                     figsize=(12,5),xlabel='Place')

In [ ]:
df_tweet_date=df_tweet.set_index('created_at')

In [ ]:
df_tweet_date.resample('D').mean()[['polarity','subjectivity']].dropna().plot(figsize=(10,6),xlabel='Date,time')

### uni-variate Analysis on Hashtags


In [ ]:
hashtag_df=df_tweet[['original_text','hashtags','retweet_hashtags']]

In [ ]:
def find_hashtags(df_tweet):
    '''This function will extract hashtags'''
    return re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', df_tweet)

hashtag_df['hashtag_check']=df_tweet.original_text.apply(find_hashtags)

In [ ]:
hashtag_df.dropna(subset=['hashtag_check'],inplace=True)

In [ ]:
tags_list=list(hashtag_df['hashtag_check'])

In [ ]:
hashtags_list_df = pd.DataFrame([tag for tags_row in tags_list for tag in tags_row],columns=['hashtag'])

In [ ]:
hashtags_list_df

In [ ]:
_=hashtags_list_df.value_counts()[:10].plot(kind='bar',figsize=(12,5),xlabel='Actual Hashtags')

In [ ]:
#Convert hastags to lowercase
hashtags_list_df['hashtag'] = hashtags_list_df['hashtag'].str.lower()

In [ ]:
_=hashtags_list_df.value_counts()[:10].plot(kind='bar',figsize=(12,5),xlabel='Hashtags in Lowercase')

In [ ]:
user_summary=df_tweet.groupby('original_author').agg({'cleaned_text':'count','followers_count':'max',
                                         'polarity':'mean','subjectivity':'mean', 'sentiment':pd.Series.mode})

In [ ]:
user_summary.sort_values(by='cleaned_text',ascending=False).head(10)

Sentiment summary of the tweets

In [ ]:
text_grouped = df_tweet.groupby('sentiment').count()['cleaned_text'].reset_index()

In [ ]:
text_grouped

In [ ]:
sns.countplot(x='sentiment', data=df_tweet)

**Most frequent words in our tweets dataset**

In [ ]:
english_tweets=cleaner.remove_other_languages_tweets(df_tweet)

In [ ]:
# text Preprocessing
english_tweets['cleaned_text'] = english_tweets['cleaned_text'].str.lower()
english_tweets['cleaned_text'] = english_tweets['cleaned_text'].apply(
    lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
all_words=' '.join(english_tweets.cleaned_text.values)
wordcloud_obj=WordCloud(width=1000,height=600,stopwords=STOPWORDS).generate(all_words)

In [ ]:
plt.figure(figsize=(20, 10))

fgg=plt.imshow(wordcloud_obj)
plt.axis('off')
# plt.title('Most Frequent Words In Our Tweets',fontsize=16)
plt.show()

In [ ]:
fgg.figure.savefig('sql_dashboard/cw_rdf.png',bbox_inches='tight',pad_inches=0)

### Topic modeling

In [ ]:
def preprocess_data(tweets_df):
    # Converting tweets to list of words For feature engineering
    sentence_list = [tweet for tweet in tweets_df['cleaned_text']]
    word_list = [sent.split() for sent in sentence_list]
    #Save only words and excludes emojis, punctuations
    word_list_new=[]
    for sent in word_list:
        word_list_new.append([re.split(r'\W+',word) for word in sent if word not in STOPWORDS and not word.isdigit()])
    
    word_list_final=[]
    for sent in word_list_new:
        word_list_final.append([i[0] for i in sent])
    # Create dictionary which contains Id and word 
    word_to_id = corpora.Dictionary(word_list_final)
    corpus_1 = [word_to_id.doc2bow(tweet) for tweet in word_list_final]

    return word_list_final, word_to_id, corpus_1

In [ ]:
word_list, id2word, corpus=preprocess_data(english_tweets)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=False)

In [ ]:
pprint(lda_model.show_topics(formatted=False))

### Model Analysis

In [ ]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# doc_lda = lda_model[corpus]

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)

**Model 1 words with some digits, 5 topics**

Perplexity:  -9.857909007134007\
Ldamodel Coherence Score/Accuracy on Tweets:  0.4242327533406264

**Model 2 words without digits, 5 topics**

Perplexity:  -9.82031321033761\
Ldamodel Coherence Score/Accuracy on Tweets:  0.4368570452021986

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

**Number of Topics optimatization**

As we can see the coherence accuracy increases with number of topics which is expected but again \
many topics again would lead to meaningless conclusion. Let's use elbow method to find optimum number of topics

In [ ]:
# Find best LDA Model considering different number of topics

params_dic={'topics_number':[],'coherence':[],'perplexity':[]}
for top_number in range(1,10):
    lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                            id2word=id2word,
                                            num_topics=top_number, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=False)
    
    perplexity=lda_model.log_perplexity(corpus)  
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    params_dic['topics_number'].append(top_number)
    params_dic['coherence'].append(coherence_lda)
    params_dic['perplexity'].append(perplexity)

In [ ]:
pd.DataFrame(params_dic).plot(y=['coherence'])
# pd.read_csv("elbow_metrics.csv",index_col=0).plot(y=['coherence'])

###  END

In [ ]:
import numpy as np

In [ ]:
alphas = list(np.arange(0.01, 3, 0.1))
alphas.append('symmetric')
alphas.append('asymmetric')
alphas.append('auto')

In [ ]:
# Find best LDA Model considering different number of alpha

params_dic_alpha={'alpha':[],'coherence':[],'perplexity':[]}
for alpha in alphas:
    lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                            id2word=id2word,
                                            num_topics=7, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha=alpha,
                                            per_word_topics=False)
    
    perplexity=lda_model.log_perplexity(corpus)  
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    params_dic_alpha['alpha'].append(alpha)
    params_dic_alpha['coherence'].append(coherence_lda)
    params_dic_alpha['perplexity'].append(perplexity)

In [ ]:
pd.DataFrame(params_dic_alpha).plot(y='coherence')#.sort_values('coherence',ascending=False)

**Model with selected optimum parameters**

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.1,
                                           per_word_topics=False)

In [ ]:
pprint(lda_model.show_topics(formatted=False))

In [ ]:
# Compute Perplexity
#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# doc_lda = lda_model[corpus]

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared